<a href="https://colab.research.google.com/github/SY-256/anomaly_detection/blob/main/notebook/chapter5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 二項分布の最尤推定による異常検知

## モデルの学習

二項分布の発生率（成功確率）パラメータμの推定値$\hat{\mu}$を最尤推定で求める

※「**すべての選手が同じ成功確率**$\mu$**を持つという仮定**（＝過分散なし）」を置くことに相当し、最尤推定で求められる成功確率パラメータ$\hat{\mu}$は、全選手で共通と仮定したスリーポイントシュートの成功確率にあたる。

In [ ]:
# 二項分布の最尤推定による異常検知の実装例（学習）
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

#### 学習データの読み込みと前処理
# スリーポイントシュート成功データ
df_train = pd.read_csv("https://raw.githubusercontent.com/ghmagazine/python_anomaly_detection_book/refs/heads/main/notebooks/datasets/player_stats_2022.csv")
df_train = df_train[df_train["FG3A"] > 0] # 試行回数0を除外
x_train = df_train["FG3M"].to_numpy() # 成功回数（発生数）
n_train = df_train["FG3A"].to_numpy() # 試行回数

#### 学習ステップ1. 正常のモデルを作成する ####
mu = np.sum(x_train) / np.sum(n_train) # 学習データ全体での発生率（発生率パラメータμの最尤推定）

#### 学習ステップ2. 異常を表す指標（異常度）を定義する ####
# 式のみで定義

#### 学習ステップ3. 異常度に閾値を設ける ####
# 推論時に都度算出するので、学習時は閾値を設定しない

#### 学習で求めたパラメータを表示 ####
print(f"mu={mu}")

ここで求めた`mu`が、発生率パラメータの最尤推定量$\hat{\mu}$になる。学習データ（正常データ）に関して、試行数$n$を横軸、成功率$\frac{x}{n}$を縦軸に取った散布図をプロット

In [ ]:
# 試行数nと成功率x/nのプロット
import seaborn as sns

# 成功率
mu_train = x_train / n_train
# 描画用のFigureとAxesの生成
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 5))
# 分母nと成功率x/nの関係を描画
sns.scatterplot(x=n_train, y=mu_train, c="#999999", ax=ax)
ax.set_xlabel("n (FG3A)", fontsize=12)
ax.set_ylabel("x/n (3P%)", fontsize=12)
# グラフを表示
plt.show()

試行回数$n$が小さいほど成功率$\frac{x}{n}$のばらつきが大きく、平均から外れた値が観測されやすい。よって成功率$\frac{x}{n}$に閾値を設けてしまうと、$n$が小さい選手ほど誤報が増える。そのため、$n$に合わせて閾値を変化させる必要があり、それを実現するためには発生率パラメータの最尤推定量$\hat{\mu}$と二項分布モデルを用いて、推論時に閾値を動的に決める必要がある。

## 推論

学習フェーズで推定したパラメータ$\hat{\mu}$と推論データの試投数$n$を用いて、二項分布モデルに基づく上下の閾値$x_{thh}$, $x_{thl}$を求める

In [ ]:
# 二項分布の最尤推定による異常検知の実装例（推論）
import pandas as pd

#### 学習したパラメータを記載 ####
MU = mu # 発生率パラメータμ

### 推論データの読み込みと前処理 ####
# 2023年度のスリーポイントシュート成功率データを推論データとする
df_inference = pd.read_csv("https://raw.githubusercontent.com/ghmagazine/python_anomaly_detection_book/refs/heads/main/notebooks/datasets/player_stats_2023.csv")
# 試投数5以下のデータを除外（表示の都合上）
df_inference = df_inference[df_inference["FG3A"] > 5]
n_inference = df_inference["FG3A"].to_numpy() # 推論対象の試投数n
x_inference = df_inference["FG3M"].to_numpy() # 推論対象の成功数x

#### 推論を実行 ####
TARGET_FP_RATE = 0.0027 # ターゲットとする誤報率（正規分布の3σ相当=0.0027）
# 下側閾値（二項分布の累積分布関数から計算）
th_lowers = stats.binom.ppf(TARGET_FP_RATE/2, n=n_inference, p=MU)
# 上側閾値（二項分布の累積密度関数から計算）
th_uppers = stats.binom.ppf(1 - TARGET_FP_RATE/2, n=n_inference, p=MU)
# 閾値により異常の有無を判定
pred = np.where((x_inference < th_lowers) | (x_inference > th_uppers), "anomaly", "normal")
# 推論結果を表示（閾値を超えた選手）
df_inference["3P%"] = x_inference / n_inference # 成功率
df_inference["prediction"] = pred
print(df_inference[df_inference["prediction"] == "anomaly"][
    ["PLAYER_ID", "3P%", "FG3A", "FG3M"]
].sort_values("3P%", ascending=False).reset_index(drop=True))

検出された選手は前年度のリーグ平均と比較して、偶然の範囲を超えてスリーポイントシュートの成功率が高い、あるいは低いと言える。

In [ ]:
from scipy.spatial import distance
# 正常と異常の境界をプロット
import matplotlib.cm as cm
from sklearn.neighbors import NearestNeighbors

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))

#### 正常と異常の範囲の色分け ####
# (x, y)格子点を作成
n_list = np.arange(1, 751)
n_grid = np.array(sum([[n for x in range(n+1)] for n in n_list], []))
x_grid = np.array(sum([[x for x in np.arange(n+1)] for n in n_list], []))
mu_grid = x_grid / n_grid # 観測された発生率μ'=x/n
# 下側閾値（二項分布の累積分布関数から計算）
th_lowers = stats.binom.ppf(TARGET_FP_RATE/2, n=n_grid, p=MU)
# 上側閾値（二項分布の累積分布関数から計算）
th_uppers = stats.binom.ppf(1 - TARGET_FP_RATE/2, n=n_grid, p=MU)
# 閾値判定
pred_grid = np.where((x_grid < th_lowers) | (x_grid > th_uppers), 0, 1)
# (n, μ)グリッドに判定を近似的に割り当てる
mu_list = np.linspace(0, 1, 500)
X1, X2 = np.meshgrid(n_list, mu_list)
X_neighbor = np.c_[X1.ravel(), X2.ravel()]
knn = NearestNeighbors(n_neighbors=1, algorithm="auto", metric="minkowski")
knn.fit(np.c_[n_grid, mu_grid])
distances, indices = knn.kneighbors(X_neighbor)
pred_neighbor = np.array([
    pred_grid[i]
    for i in indices
])
# 正常と異常の境界をプロット
pred_pivot = pred_neighbor.reshape(X1.shape)
ax.contourf(X1, X2, pred_pivot, cmap=cm.gray, alpha=0.5)

#### 各データを散布図としてプロット ####
sns.scatterplot(df_inference, x="FG3A", y="3P%", hue="prediction",
                palette=["#999999", "#111111"], ax=ax)
ax.set_xlim(0, 750)
ax.legend()
# グラフ表示
plt.show()

試投回数$n$が少ない選手は偶然の影響で高い（または低い）成功率を記録しやすいため、閾値を$n$に応じて調整する必要がある

二項分布の累積分布関数に基づく閾値を用いるため、$n$に応じた分布の変化に合わせて動的に調整できていることがわかる。この方法によって、誤報率を一定に保ったまま、偶然の範囲を超えて高い、または低い成功率を記録した選手を検出できる。

# ポアソン分布の最尤推定による異常検知

計数データのうち、連続的な観測区間$t$あたりの発生数$x$に異常検知を適用する場合、最尤推定したポアソン分布を正常のモデルとして用いた、教師なし学習による異常検知手法が有名

典型例として、時間あたりの故障発生数に異常検知を適用するケースが挙げられる。経過時間を$t$で表したとき、その時間内での故障発生数$x$は、ポアソン分布$Po(x \mid \mu t)$に従う。

In [ ]:
# ポアソン分布の最尤推定による異常検知の実装例（学習）
import pandas as pd
import numpy as np
from scipy import stats

##### 学習データの読み込みと前処理 #####
# 日本の活火山データ
df = pd.read_csv("https://raw.githubusercontent.com/ghmagazine/python_anomaly_detection_book/refs/heads/main/notebooks/datasets/volcano_prefectures.csv")
df_train = df[df["region"].isin(["Hokkaido", "Tohoku", "Kanto", "Chubu"])] # 対象地域を絞る
t_train = df_train["area"].to_numpy() # 面積t
x_train = df_train["volcanoes"].to_numpy() # 活火山の数x

##### 学習ステップ1. 正常のモデルを作成する #####
mu = np.sum(x_train) / np.sum(t_train) # 面積あたりの平均発生数パラメータμの最尤推定量

##### 学習ステップ2. 異常を表す指標（異常度）を定義する #####
# 式を定義するのみでプログラム上は実行しない

##### 学習ステップ3. 異常度に閾値を設ける #####
# 推論時に都度算出するので、学習時は閾値を求めない

##### 学習で求めたパラメータを表示 #####
print(f"mu={mu}")

ここで求めた`mu`が、単位時間当たり平均発生数の最尤推定量$\hat{\mu}$となる。学習データに関して、面積$t$を横軸、面積当たりの活火山数$\frac{x}{t}$を縦軸にとった散布図をプロットしてみる。

In [ ]:
# 面積tと面積あたりの活火山数x/tの関係をプロット
import matplotlib.pyplot as plt
import seaborn as sns

# 発生数
ration_train = x_train / t_train
# 描画用
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 5))
# 分母nと発生数x/nの関係を描画
sns.scatterplot(x=t_train, y=ration_train, c="#999999", ax=ax)
ax.set_xlabel("t (area)", fontsize=12)
ax.set_ylabel("x/t (volcanoes) / area", fontsize=12)
# グラフ表示
plt.show()

面積$t$が小さいほど面積あたりの活火山数$\frac{x}{t}$のばらつきが大きく、平均から外れた値が観測されやすいことがわかる。よって面積あたりの活火山数$\frac{x}{t}$に閾値を設けてしまうと、$t$が小さい都道府県ほど誤報が増える。そのため、$t$に合わせて閾値を変更する必要があり、それを実現するために$\hat{\mu}$とポアソン分布モデルを用いて、推論時に閾値を決定する。

In [ ]:
# ポアソン分布の最尤推定による異常検知の実装例（推論）
import pandas as pd

##### 学習したパラメータを記載
MU=mu # 面積あたり平均発生数パラメータμ

##### 推論データの読み込みと前処理 #####
# 日本の活火山データ
df = pd.read_csv("https://raw.githubusercontent.com/ghmagazine/python_anomaly_detection_book/refs/heads/main/notebooks/datasets/volcano_prefectures.csv")
df_inference = df[~df["region"].isin(["Hokkaido", "Tohoku", "Kanto", "Chubu"])].copy() # 対象地域を絞る（学習に使用していない地域）
t_inference = df_inference["area"].to_numpy() # 面積t
x_inference = df_inference["volcanoes"].to_numpy() # 活火山の数x

##### 推論を実行 #####
TARGET_FP_RATE = 0.0027 # ターゲットとする誤報率（正規分布の3σ相当=0.0027）
# 下側閾値（ポアソン分布の累積分布関数から計算）
th_lowers = stats.poisson.ppf(TARGET_FP_RATE/2, mu=MU * t_inference)
# 上側閾値（ポアソン分布の累積分布関数から計算）
th_uppers = stats.poisson.ppf(1 - TARGET_FP_RATE/2, mu=MU * t_inference)
# 閾値により異常の有無を判定
pred = np.where((x_inference < th_lowers) | (x_inference > th_uppers),
                "anomaly", "normal")
# 推論結果を表示（閾値を超えた都道府県）
df_inference["volcanoes / area"] = x_inference / t_inference # 面積あたりの発生数
df_inference["prediction"] = pred
print(df_inference[df_inference["prediction"] == "anomaly"][
    ["prefecture", "volcanoes / area", "volcanoes", "area"]].sort_values(
        "volcanoes / area", ascending=False
    ).reset_index(drop=True))

異常判定の閾値を超えた都道府県（鹿児島県）は、火山島（陸地面積をあまり増やさずに活火山数が増える）が多いというドメイン知識があり、これが面積あたりの活火山数の増加に寄与したと想定される。